In [1]:
import configparser
import os

In [2]:
# Create environment variable for OpenAI API key
configparser = configparser.ConfigParser()
configparser.read("config.ini")
os.environ["OPENAI_API_KEY"] = configparser["keys"]["openai"]
os.environ["ACTIVELOOP_TOKEN"] = configparser["keys"]["activeloop"]

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [5]:
# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [6]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [8]:
# create Deep Lake dataset
my_activeloop_org_id = "fabiogomez11c"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


In [9]:
# add documents to our Deep Lake dataset
db.add_documents(docs)

/

Dataset(path='hub://fabiogomez11c/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['7b461fa0-1a01-11ee-870a-f42679ade418',
 '7b462108-1a01-11ee-870a-f42679ade418']

In [10]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [11]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [12]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [13]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://fabiogomez11c/langchain_course_from_zero_to_hero already exists, loading from the storage


\

Dataset(path='hub://fabiogomez11c/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['74407272-1a02-11ee-870a-f42679ade418',
 '7440740c-1a02-11ee-870a-f42679ade418']

In [14]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [15]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


In [16]:
response = agent.run("When was Lewis Hamilton born?")
print(response)



> Entering new  chain...
 I need to find out when Lewis Hamilton was born.
Action: Retrieval QA System
Action Input: When was Lewis Hamilton born?
Observation:  I don't know.
Thought: I should look up the answer online.
Action: Retrieval QA System
Action Input: What is the date of birth of Lewis Hamilton?
Observation:  I don't know.
Thought: I should search for the answer.
Action: Retrieval QA System
Action Input: What is the birthdate of Lewis Hamilton?
Observation:  I don't know.
Thought: I should search for the answer in a reliable source.
Action: Retrieval QA System
Action Input: Where can I find the birthdate of Lewis Hamilton?
Observation:  I don't know.
Thought: I should search for the answer in a reliable source.
Action: Retrieval QA System
Action Input: What is the birthdate of Lewis Hamilton according to Wikipedia?
Observation:  I don't know.
Thought: I should search for the answer in a reliable source.
Action: Retrieval QA System
Action Input: What is the birthdate of Lewi